In [1]:
#pragma cling add_include_path("/usr/include/CGAL")
#pragma cling add_library_path("/usr/lib/x86_64-linux-gnu")

#pragma cling load("gmp")
#pragma cling load("mpfr")
#pragma cling load("CGAL")

In [2]:
#include <type_traits>
#include <iostream>
#include <algorithm>
#include <vector>
#include <cassert>
#include <cstdlib>
#include <fstream>
#include <iostream>
#include <memory>
#include <sstream>
#include <string>
#include <string_view>
#include <utility>
#include <vector>


#include <CGAL/Exact_predicates_inexact_constructions_kernel.h>
#include <CGAL/Origin.h>
#include <CGAL/Polyhedron_3.h>
#include <CGAL/Tetrahedron_3.h>
#include <CGAL/Triangle_3.h>
#include <CGAL/Triangulation_3.h>
#include <CGAL/Delaunay_triangulation_3.h>
#include <CGAL/Triangulation_vertex_base_with_info_3.h>
#include <CGAL/box_intersection_d.h>
#include <CGAL/convex_hull_3.h>

In file included from input_line_8:15:
In file included from /usr/include/CGAL/Exact_predicates_inexact_constructions_kernel.h:28:
In file included from /usr/include/CGAL/Simple_cartesian.h:28:
In file included from /usr/include/CGAL/Cartesian/Cartesian_base.h:28:
In file included from /usr/include/CGAL/basic.h:44:
In file included from /usr/include/CGAL/number_type_basic.h:48:
In file included from /usr/include/CGAL/Rational_traits.h:31:
In file included from /usr/include/CGAL/is_convertible.h:28:
/home/pbarletta/anaconda3/include/gmpxx.h:210:6: error: use of undeclared identifier 'DBL_MAX_EXP'
  {  __GMPXX_TMPZ_D;    mpz_add (z, w, temp); }
     ^
/home/pbarletta/anaconda3/include/gmpxx.h:121:19: note: expanded from macro '__GMPXX_TMPZ_D'
  mp_limb_t limbs[__GMPZ_DBL_LIMBS];                                    \
                  ^
/home/pbarletta/anaconda3/include/gmpxx.h:79:48: note: expanded from macro '__GMPZ_DBL_LIMBS'
#define __GMPZ_DBL_LIMBS __GMPXX_BITS_TO_LIMBS(DBL_MAX_EXP)+1

Interpreter Error: 

In [5]:
struct VertexInfo {
public:
    VertexInfo() = default;

    VertexInfo(int const index, double const radius, int const resn,
        std::string_view const resi) :cd
        _index(index),
        _radius(radius), _resn(resn), _resi(resi) {}

    int _index;
    double _radius;
    // atom's residue number
    int _resn;
    // atom's residue name in 3 letter format
    std::string _resi;
};

// clang-format off
// Basic definitions
using EPIC = CGAL::Exact_predicates_inexact_constructions_kernel;
using Vb = CGAL::Triangulation_vertex_base_with_info_3<VertexInfo, EPIC>;
using Tds = CGAL::Triangulation_data_structure_3<Vb>;
using Delaunay = CGAL::Delaunay_triangulation_3<EPIC, Tds>;

// Definitions for Delaunay triangulation
using CVector = CGAL::Vector_3<EPIC>;
using Segment = CGAL::Segment_3<EPIC>;
using CPoint = Delaunay::Point;
using Vertex_handle = Delaunay::Vertex_handle;
using Cell_handle = Delaunay::Cell_handle;
using Vertex_iterator = Delaunay::Vertex_iterator;
using Finite_vertices_iterator = Delaunay::Finite_vertices_iterator;
using All_vertices_iterator = Delaunay::All_vertices_iterator;
using Edge_iterator = Delaunay::Edge_iterator;
using Facet_iterator = Delaunay::Facet_iterator;
using Finite_facets_iterator = Delaunay::Finite_facets_iterator;
using Cell_iterator = Delaunay::Cell_iterator;
using Finite_cells_iterator = Delaunay::Finite_cells_iterator;
using All_cells_iterator = Delaunay::All_cells_iterator;
using Cell_circulator = Delaunay::Cell_circulator;

// Definitions for convex hull
using Polyhedron = CGAL::Polyhedron_3<EPIC>;
using P_Facet_iterator = Polyhedron::Facet_iterator;
using P_Facet_const_iterator = Polyhedron::Facet_const_iterator;
using P_Edge_iterator = Polyhedron::Edge_iterator;
using P_Edge_const_iterator = Polyhedron::Edge_const_iterator;
using P_Halfedge_around_facet_circulator =
    Polyhedron::Halfedge_around_facet_circulator;
using P_Halfedge_around_facet_const_circulator =
    Polyhedron::Halfedge_around_facet_const_circulator;
using P_Vertex_iterator = Polyhedron::Vertex_iterator;
using P_Vertex_const_iterator = Polyhedron::Vertex_const_iterator;

// Miscellaneous definitions
using Object = CGAL::Object;
using Triangle = CGAL::Triangle_3<EPIC>;
using Triang_Vector = std::vector<Triangle>;
using Segment = EPIC::Segment_3;
using Tetrahedron = CGAL::Tetrahedron_3<EPIC>;
using Tetra_Vector = std::vector<Tetrahedron>;
using Box =
    CGAL::Box_intersection_d::Box_with_handle_d<double, 3, Finite_cells_iterator>;

// ANA definitions
using MD_Element = std::array<CPoint, 4>;
// Cell
using MD_Vector = std::vector<MD_Element>;
// Pocket
using MD_Matrix = std::vector<MD_Vector>;
// All voids
using NDD_Element = std::array<std::pair<CPoint, double>, 4>;
// Cell
using NDD_Vector = std::vector<NDD_Element>;
// Pocket
using NDD_Matrix = std::vector<NDD_Vector>;
// All voids
using NDD_IElement = std::array<int, 4>;
// Cell indices
using NDD_IVector = std::vector<NDD_IElement>;
// Pocket indices
using NDD_IMatrix = std::vector<NDD_IVector>;
// All voids indices
using NA_Vector = std::vector<Finite_cells_iterator>;
// Pocket
using NA_Matrix = std::vector<NA_Vector>;
// All voids
using Poly_Vector = std::vector<Polyhedron>;
// Pocket border cells
using Poly_Matrix = std::vector<Poly_Vector>;
// All null areas border cells
using ANA_molecule = std::vector<std::pair<CPoint, VertexInfo>>;

In [6]:

    class Modes {
    public:
        Modes(std::string const &modes_filename);

        void get_modes_from_raw(std::string_view const texto);

        std::vector<std::vector<double>> _atm_evectors;
        std::vector<std::vector<double>> _evectors;
        std::vector<double> _evals;
        size_t _i, _j;
    };

In [7]:


    void Modes::get_modes_from_raw(std::string_view const texto) {

        size_t beg = 57;
        if (isspace(texto[38])) {
            std::string_view st_j(texto.data() + 34, 3);
            _j = std::stoi(st_j.data());
            std::string_view st_i(texto.data() + 49, 3);
            _i = std::stoi(st_i.data());
        } else {
            std::string_view st_j(texto.data() + 34, 4);
            _j = std::stoi(st_j.data());
            std::string_view st_i(texto.data() + 48, 4);
            _i = std::stoi(st_i.data());
            ++beg;
        }
        _evectors.reserve(_j);
        _evals.reserve(_j);
        
        size_t constexpr ch_elem = 11;
        size_t constexpr ch_num = 5;
        size_t constexpr ch_eval = 12;
        size_t constexpr ncolumns = 7;
        size_t const resto = _i % ncolumns;
        size_t const nlines = (resto) ? _i / ncolumns + 1 : _i / ncolumns;
        // representa: " ****"
        size_t constexpr spacer = 6;
        // número de caracteres q ocupa 1 modo en el formato de Amber.
        size_t const ch_mode = (nlines - 1) * ncolumns * ch_elem +
            resto * ch_elem + nlines + spacer;

        const char *cursor = texto.data() + beg + ch_mode;
        for (size_t K = 0; K < _j; ++K) {
            char const *start_eval = cursor + ch_num;
            char *end_eval = const_cast<char *>(start_eval) + ch_eval;
            _evals.push_back(std::strtof(start_eval, &end_eval));

            auto idx = cursor + ch_num + ch_eval;
            size_t nl = 0;
            std::vector<double> temp;
            temp.reserve(_i);
            // nl is the new line counter. At the beginning, idx is pointing
            // to the new line character after the eigenvalue and before the
            // first element of the eigenvector, and since 0 % 7 evalueates
            // to false, nl starts is incremented in the first iteration.
            for (size_t k = 0; k < _i; ++k) {
                (k % 7) ? nl = 0 : nl = 1;
                char const *start_elem = idx;
                char *end_elem = const_cast<char *>(start_elem) + ch_elem;
                temp.push_back(std::strtof(start_elem, &end_elem));
                idx = idx + ch_elem + nl;
            }
            _evectors.push_back(std::move(temp));
            cursor = cursor + ch_mode + ch_num + ch_eval + 1;
        }

        return;
    }

In [8]:
 Modes::Modes(std::string const &modes_filename) {

        std::ifstream ifs(modes_filename);
        if (ifs.is_open()) {
            ifs.seekg(0, ifs.end);
            size_t const fsz = static_cast<size_t>(ifs.tellg());
            std::unique_ptr<char[]> const buffer =
                std::make_unique<char[]>(fsz);
            ifs.seekg(0);
            ifs.read(buffer.get(), fsz);
            ifs.close();

            get_modes_from_raw(std::string_view(buffer.get(), fsz));
        } else
            std::cout << "Could not get vectors from: " << modes_filename << '\n';
    }

In [9]:
Modes modo("/home/pbarletta/labo/18/lbp/run/4xcp/ndd/tmp/modes_4xcp")

In [13]:
double sum = 0.;
for (auto const each : modo._evectors[1])
    sum += each*each;

In [17]:
modo._evectors[0]

{ -0.01876, -7e-05, 0.01312, -0.01975, 0.00426, 0.01314, -0.02428, 0.01113, 0.01754, -0.01465, 0.01109, 0.01285, -0.00847, 0.01599, 0.01121, -0.00984, 0.02338, 0.00816, -0.01291, 0.03105, 0.00631, 0.00072, 0.03698, 0.00324, 0.00669, 0.02212, 0.00295, -0.00551, 0.01821, 0.00585, -0.00287, 0.03112, 0.0045, 0.01387, 0.02669, -0.00034, 0.0122, 0.01036, 0.00298, 0.00311, 0.01271, 0.00646, 0.01587, 0.02226, -0.00198, 0.02887, 0.01184, 0.0008, 0.04562, 0.00458, 0.00246, 0.04694, 0.00707, 0.00662, 0.04214, -0.00202, -0.00951, 0.02808, 0.00538, -0.01288, 0.025, 0.01818, -0.00127, 0.02175, 0.0165, -0.00314, 0.01102, 0.0081, -0.00929, 0.00739, 0.017, -0.00512, 0.005, 0.02757, -0.0001, -0.00491, 0.01945, -0.00629, -0.01321, 0.01714, -0.00894, -0.01342, 0.0314, -0.00386, -0.01971, 0.03519, -0.00324, -0.03268, 0.02579, -0.01065, -0.03178, 0.02337, -0.0095, -0.03682, 0.03602, -0.0058, -0.04067, 0.03287, -0.0103, -0.05183, 0.04165, -0.01253, -0.04138, 0.03649, -0.0095, -0.02211, 0.03014, -0.00658, -0.

In [24]:
sasa._vxyz[5]

0